In [2]:
# plot the audio data
# use popup window
%matplotlib qt
import matplotlib.pyplot as plt

from utils import *

chrip freq 5000, 10000.0
chrip freq 7000, 11000.0


In [3]:
import pickle

In [ ]:
# line delay

rate = 48000

empty = np.zeros(int(0.5 * rate))

duration = 0.1
t = np.linspace(0, duration, int(rate * duration)+1)[:-1]
signal = sine_440hz(t)

record, play, _ = play_and_record(np.concatenate((signal, empty, signal, empty)), 48000)

# pickle.dump(record, open('line_delay_record.pkl', 'wb'))
# pickle.dump(play, open('line_delay_play.pkl', 'wb'))

In [15]:
record = pickle.load(open('line_delay_record.pkl', 'rb'))
play = pickle.load(open('line_delay_play.pkl', 'rb'))

x_range = np.arange(0, len(record)) / rate
plt.plot(x_range, record)
plt.plot(x_range, play)

qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x61c510497010


In [65]:
# ask line coding

rate = 48000

one = np.array([0, 1])
zero = np.array([0, -1])

signal = np.concatenate((one, zero, zero, one, one))

record, _, _ = play_and_record_precise2(signal, rate)

for i in range(len(signal) // 2):
    plt.plot([i*2, i*2+1], [record[i*2], record[i*2+1]])
    plt.plot([i*2, i*2+1], [signal[i*2], signal[i*2+1]])

# plt.plot(signal)
# plt.plot(record)
plt.show()

The cached device pixel ratio value was stale on window update.  Please file a QTBUG which explains how to reproduce.
qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x56ef06773e70


In [66]:
# pickle.dump(record, open('line_coding_record.pkl', 'wb'))
# pickle.dump(signal, open('line_coding_signal.pkl', 'wb'))

In [73]:
pickle.load(open('line_coding_record.pkl', 'rb'))
pickle.load(open('line_coding_signal.pkl', 'rb'))

# plt.plot(record)
# plt.plot(signal)

# instead of line, we plot dots
# plt.plot(record, '-o')
# plt.plot(signal, '-o')

signal_color = 'orange'
record_color = 'blue'

for i in range(len(signal) // 2):
    plt.plot([i*2, i*2+1], [record[i*2], record[i*2+1]], '-o', color='C0')
    plt.plot([i*2, i*2+1], [signal[i*2], signal[i*2+1]], '-o', color='C1')

# plt.plot(signal)
# plt.plot(record)
plt.show()

The cached device pixel ratio value was stale on window update.  Please file a QTBUG which explains how to reproduce.
qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x56ef05944f00


In [9]:
# air_flip

rate = 48000

empty = np.zeros(int(0.5 * rate))

duration = 0.2
t = np.linspace(0, duration, int(rate * duration)+1)[:-1]
signal = sine_wave(1000)(t)


sine_wav = sine_wave_samples(440, 1 * 440, 0, 1, rate)
# empty for 0.2s
empty = np.zeros(int(0.2 * rate))
audio = np.concatenate((sine_wav, empty, chirp1, signal, np.zeros(int(0.5*rate))))

record_wave, play_wave, rate = play_and_record(audio, rate, discard=False)

magic_correction = 0

chirp1_loc = locate_chirp(record_wave, chirp1) + magic_correction
# chirp2_loc = locate_chirp(record_wave, chirp2) + magic_correction
# chirp2_loc = min(chirp2_loc, len(record_wave))

start = chirp1_loc + 1
end = start + len(signal)

play = np.concatenate((chirp1, signal))
record = record_wave[start - len(chirp1):end]

pickle.dump(record, open('air_flip_record.pkl', 'wb'))
pickle.dump(play, open('air_flip_play.pkl', 'wb'))

qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x61c510bf7d10


In [16]:
pickle.load(open('air_flip_record.pkl', 'rb'))
pickle.load(open('air_flip_play.pkl', 'rb'))

x_axis = np.arange(0, len(record)) / rate
plt.plot(x_axis, play)
plt.plot(x_axis, record)

In [14]:
# air_drift

rate = 48000

empty = np.zeros(int(0.5 * rate))

duration = 0.2
t = np.linspace(0, duration, int(rate * duration)+1)[:-1]
signal = sine_wave(2000)(t)


sine_wav = sine_wave_samples(440, 1 * 440, 0, 1, rate)
# empty for 0.2s
empty = np.zeros(int(0.2 * rate))
audio = np.concatenate((sine_wav, empty, chirp1, signal, np.zeros(int(0.5*rate))))

record_wave, play_wave, rate = play_and_record(audio, rate, discard=False)

magic_correction = 0

chirp1_loc = locate_chirp(record_wave, chirp1) + magic_correction
# chirp2_loc = locate_chirp(record_wave, chirp2) + magic_correction
# chirp2_loc = min(chirp2_loc, len(record_wave))

start = chirp1_loc + 1
end = start + len(signal)

play = signal
record = record_wave[start:end]

record = -record

pickle.dump(record, open('air_drift_record.pkl', 'wb'))
pickle.dump(play, open('air_drift_play.pkl', 'wb'))

# plt.plot(play)
# plt.plot(record)

qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x61c510b4c670


In [7]:
# fsk

ENABLE_PAUSE = 0
PAUSE_TIME_RATIO = 0.5
ENABLE_SHIFT_CORR = 0

def fsk_modulation(binary_data, rate, symbol_freq, fsk_freq):
    symbol_samples = int(rate / symbol_freq)
    pause_samples = int(symbol_samples * PAUSE_TIME_RATIO) if ENABLE_PAUSE else 0
    symbol_time = 1 / symbol_freq

    print(f"Min FSK freq: {symbol_freq}")
    assert fsk_freq >= symbol_freq

    t = np.linspace(0, symbol_time, symbol_samples+1)[:-1]
    one = sine_wave(fsk_freq)(t)
    zero = sine_wave(2*fsk_freq)(t)
    one = np.concatenate((one, np.zeros(pause_samples)))
    zero = np.concatenate((zero, np.zeros(pause_samples)))

    fsk_wave = np.concatenate([one if bit == 1 else zero for bit in binary_data])
    return fsk_wave

def simple_fsk_demodulation(wave, freq):
    rate = 48000
    symbol_samples = int(rate / freq)

    symbol_time = 1 / freq

    t = np.linspace(0, symbol_time, symbol_samples+1)[:-1]
    one = sine_wave(freq)(t)
    zero = sine_wave(2*freq)(t)

    receive_fsk_wave = wave[:len(wave) - len(wave) % symbol_samples]
    result = np.zeros(len(receive_fsk_wave) // symbol_samples)
    for i in range(len(result)):
        wave = receive_fsk_wave[i*symbol_samples:(i+1)*symbol_samples]
        one_corr = abs(np.abs(np.correlate(wave, one, 'valid')).max())
        zero_corr = abs(np.abs(np.correlate(wave, zero, 'valid')).max())
        result[i] = 1 if one_corr > zero_corr else 0
    return result
data_len = 5
# bin = np.random.randint(0, 2, size=data_len)
bin = np.array([1, 0, 0, 1, 1])

rate = 48000

symbol_freq = 1000
symbol_samples = int(rate / symbol_freq)
fsk_freq = symbol_freq
fsk_wave = fsk_modulation(bin, rate, symbol_freq, fsk_freq)

record_wave, rate, raw_record_wave = play_and_record_precise2(fsk_wave, rate)
record_wave = -record_wave
if len(record_wave) < len(fsk_wave):
    record_wave = np.concatenate((record_wave, np.zeros(len(fsk_wave) - len(record_wave))))

# receive_bits = fsk_demodulation(len(bin), record_wave.copy(), rate, symbol_freq, fsk_freq)
receive_bits = simple_fsk_demodulation(record_wave.copy(), fsk_freq)

receive_bits = receive_bits[:len(bin)]

similarity = np.sum(receive_bits == bin) / len(bin)
print(f"Similarity: {similarity}")

transfer_time = len(fsk_wave) / rate
bits = len(bin)
bps = bits / transfer_time
print(f"Transfer Time: {transfer_time} s, {bps} bps, {bps/1000} kbps")

play = fsk_wave
record = record_wave

pickle.dump(record, open('fsk_record.pkl', 'wb'))
pickle.dump(play, open('fsk_play.pkl', 'wb'))

Min FSK freq: 1000
Similarity: 1.0
Transfer Time: 0.005 s, 1000.0 bps, 1.0 kbps


In [23]:
play = pickle.load(open('fsk_play.pkl', 'rb'))
record = pickle.load(open('fsk_record.pkl', 'rb'))
fig, ax = plt.subplots()
rate = 48000
x_axis = np.arange(0, len(record)) / rate
ax.plot(x_axis, play, label='Playback')
ax.plot(x_axis, record, label='Capture')
ax.legend()
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.set_title('FSK via Air (1000Hz and 2000Hz)')
freq = 1000
symbol_samples = int(rate / freq)
for i in range(1, len(bin)):
    ax.axvline(x=(i*symbol_samples/rate), color='grey', linestyle='--')

qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x654eb9e62430
